In [1]:
import os 
import cv2
import numpy as np
from PIL import Image
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.activations import relu
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob as gb
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_path = '/kaggle/input/art-images-drawings-painting-sculpture-engraving/dataset/dataset_updated/training_set/'
test_path = '/kaggle/input/art-images-drawings-painting-sculpture-engraving/dataset/dataset_updated/validation_set'


In [3]:
# len(os.listdir(train_path) )

5

In [4]:
# size = []
# images_path=[]
# for folder_name in os.listdir(train_path):
#     folder_path = os.path.join(train_path,folder_name)
#     for image in os.listdir(folder_path):
#         try:
#             images_path.append(image)
#             img_path = os.path.join(folder_path,image)
#             img = plt.imread(img_path)
#             size.append(img.shape)
#         except:
#             pass

In [5]:
# pd.Series(size).value_counts()

(320, 240, 3)    342
(320, 320, 3)    329
(320, 427, 3)    297
(320, 480, 3)    153
(320, 213, 3)    142
                ... 
(220, 480, 3)      1
(450, 599, 3)      1
(498, 700, 3)      1
(413, 320, 3)      1
(634, 800, 3)      1
Name: count, Length: 1855, dtype: int64

In [3]:
inputt=[]
outputt=[]
test= []
def resizing(pth, gray, size=(320, 320)):
    ls_ = []
    for folder_name in os.listdir(pth):
        folder_path = os.path.join(pth, folder_name)
        for image in os.listdir(folder_path):
            img_path = os.path.join(folder_path, image)
            try:
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #RGB image 
                resize_img = cv2.resize(img, size)
                if gray:
                    gray_image = cv2.cvtColor(resize_img, cv2.COLOR_RGB2GRAY) # gray image
                    ls_.append(list(gray_image))
                else:
                    ls_.append(list(resize_img))
            except :
                pass  
    return ls_


In [4]:
inputt = resizing(train_path,gray=1)
outputt = resizing(train_path,gray=0)
testt = resizing(test_path,gray=1)

In [5]:
print(len(inputt))
print(len(outputt))
print(len(testt))

7721
7721
856


In [6]:
inputt= np.array(inputt)
outputt= np.array(outputt)
testt= np.array(testt)
inputt=np.expand_dims(inputt,axis=-1)

In [10]:
# testt.shape

(856, 320, 320)

In [11]:
# type(inputt)

numpy.ndarray

In [12]:
# sss=[]
# for img in testt:
#     sss.append(img.shape)
# set(sss)

{(320, 320)}

In [ ]:
datagen=ImageDataGenerator(rescale=1./255,
                        shear_range=0.2,
                        zoom_range=0.2,
                        rotation_range=20,
                        horizontal_flip=True)

q = datagen.flow(inputt,batch_size=64) 
k = datagen.flow(outputt,batch_size=64) 


In [11]:
def combined_generator(input_gen, output_gen):
    while True:
        X_input = input_gen.next()  # Grayscale input batch
        y_output = output_gen.next()  # RGB output batch
        yield X_input, y_output


In [13]:
def showimg (img):
    img = plt.imshow(img,cmap='gray')
    plt.axis('off')
    plt.show()

In [12]:
def u_net(h=(320,320,1)):
  gg=layers.Input(shape=h)
  x= gg
  conv =[]
  f=64

  #down_sampling
  for i in range (4):
    x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
    x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
    conv.append(x)
    x= layers.MaxPooling2D()(x)
    f*=2

  
  #bottle-neck
  x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
  x= layers.Conv2D(f,3,activation='relu',padding='same')(x)


  ff=512
  j= len(conv)-1
  #up-sampling
  for i in range(4):
    x=layers.Conv2DTranspose(ff,2,strides=(2,2),padding='same')(x)
    x=layers.Concatenate(axis=3)([x,conv[j]])
    x= layers.Conv2D(ff,3,activation='relu',padding='same')(x)
    x= layers.Conv2D(ff,3,activation='relu',padding='same')(x)
    ff//=2
    j-=1


  output = layers.Conv2D(3,1,padding='same')(x)
  model = models.Model(inputs=gg,outputs=output)
  return model

In [13]:
model = u_net()
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse'])

In [15]:
model.fit(combined_generator(input,output),batch_size=64,epochs=5)

AttributeError: 'NumpyArrayIterator' object has no attribute 'next'